In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

# Import PySpark DataFrame functions (round, col, sum etc.) with shorter alias
from pyspark.sql import functions as F

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,011 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy I

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df_sales.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2.1 Create a temporary view of the DataFrame.
df_sales.createOrReplaceTempView('HomeSales')
# 2.2 Get the data types of the columns.
df_sales.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

# 3.1. Convert the 'price' column as a float datatype and 'bedrooms' as an integer
df_sales = df_sales.withColumn("price", df_sales["price"].cast("float"))
df_sales = df_sales.withColumn("bedrooms", df_sales["bedrooms"].cast("int"))

# 3.2 Create the 'year' column from the 'date' column
df_sales = df_sales.withColumn("date", F.to_date(F.col("date"), "yyyy-MM-dd"))
df_sales = df_sales.withColumn("year", F.year(F.col("date")))

# 3.3. Register the DataFrame as a temporary view for SQL queries
df_sales.createOrReplaceTempView("HomeSales")

# 3.4 SQL query to calculate the average price for 4-bedroom houses per year
query_1 = """
SELECT year, AVG(price) AS avg_price
FROM HomeSales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year ASC
"""

# Display query results for a four bedroom house sold per year
# rounding the average price to two decimal places
spark.sql(query_1).withColumn("avg_price", F.round("avg_price", 2)).show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

# 4.1 Convert the 'date_built' column from string to integer (since it seems to contain only the year)
df_sales = df_sales.withColumn("date_built", df_sales["date_built"].cast("int"))

# 4.2 Convert the 'bathrooms' column to integer
df_sales = df_sales.withColumn("bathrooms", df_sales["bathrooms"].cast("int"))

# Display the DataFrame
df_sales.show()

+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923.0|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628.0|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866.0|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895.0|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418.0|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|20

In [7]:
# 4.3 Create a query for avg price for each year built for homes with 3 bedrooms and 3 bathrooms
query_2 = """
SELECT date_built, AVG(price) AS avg_price
FROM HomeSales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built ASC
"""
# Display query results, rounding the average price to two decimal places
spark.sql(query_2).withColumn("avg_price", F.round("avg_price", 2)).show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [8]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

# 5.1 Convert the 'sqft_living' column from string to integer
df_sales = df_sales.withColumn("sqft_living", df_sales["sqft_living"].cast("int"))

query_2 = """
SELECT date_built, AVG(price) AS avg_price
FROM HomeSales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = "2" and sqft_living >=2000
GROUP BY date_built
ORDER BY date_built ASC
"""
# Display query results, rounding the average price to two decimal places
spark.sql(query_2).withColumn("avg_price", F.round("avg_price", 2)).show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [9]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# 6.1. Record the current time to measure how long the following operations take
start_time = time.time()

# 6.2 Set query parameters
query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM HomeSales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
# 6.3 Execute the query
df_result = spark.sql(query)

# 6.4. Show the result
df_result.show()

# 6.5 Calculate runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 1.1469507217407227 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table HomeSales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('HomeSales')

True

In [14]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

# 9.1. Record the current time to measure how long the following operations take
start_time = time.time()
# 9.2 Initiate query
query_3 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM HomeSales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
# 9.3 Execute the query
df_result = spark.sql(query_3)
# 9.4. Show the result
df_result.show()
# Show run time
print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.7436661720275879 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df_sales.write.partitionBy("date_built").mode("overwrite").parquet("date_built")

In [16]:
# 11. Read the parquet formatted data in our new parquet formatted data
p_df_sales=spark.read.parquet("date_built")

In [17]:
# 12. Create a temporary table for the parquet data.
p_df_sales.createOrReplaceTempView('p_sales_p')

In [18]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

# 13.1. Record the current time to measure how long the following operations take
start_time = time.time()
# 13.2 Run and display query results
spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM p_sales_p
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
""").show()
# 13.3 Show run time
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 1.3210408687591553 seconds ---


In [22]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table HomeSales")

DataFrame[]

In [23]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("HomeSales")

False